<a href="https://colab.research.google.com/github/merriekay/S23-CS167-Notes/blob/main/Day09_Decision_Trees_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS167: Day08
## Decision Trees

#### CS167: Machine Learning, Spring 2023

Thursday, February 23rd, 2023

📆 [Course Schedule](https://docs.google.com/spreadsheets/d/e/2PACX-1vSvFV5Mz0_YZE1d5r3gQ8IMktE4cBAsJIlP30cl2GhEpSO0J-YWV62QokSDz-OcOCsEmxMuKpY0kVlR/pubhtml?gid=0&single=true) | 🙋[PollEverywhere](https://pollev.com/meredithmoore011) | 📜 [Syllabus](https://analytics.drake.edu/~moore/cs167_s23_syllabus.html) | 📬 [CodePost Login](https://codepost.io/login)

# Admin Stuff

You should be working on:
- [Notebook #3](https://classroom.github.com/a/2Jc4k7Pe) **Extension**: now due Tuesday 2/28 by 11:59pm.

# load in our data:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
path = 'datasets/restaurant.csv'  #'/content/drive/MyDrive/CS167/Datasets/restaurant.csv'
restaurant = pd.read_csv(path)

restaurant.head()

,alt,bar,fri,hun,pat,price,rain,res,type,est,target
0,Yes,No,No,Yes,Some,$$$,No,Yes,French,0-10,Yes
1,Yes,No,No,Yes,Full,$,No,No,Thai,30-60,No
2,No,Yes,No,No,Some,$,No,No,Burger,0-10,Yes
3,Yes,No,Yes,Yes,Full,$,No,No,Thai,10-30,Yes
4,Yes,No,Yes,No,Full,$$$,No,Yes,French,>60,No


# Quick Review:

## What is a decision tree?
<div>
<img src="images/day05_dt_ex1.png" width=800/>
</div>

[Decision Tree Example Slides](https://docs.google.com/presentation/d/1mQxycmE2gjC5i-Iwphpd_KAsaFxVDzJXkg_C2EBFGUQ/edit?usp=sharing)

## Decision Tree Example:
<div>
<img src="images/day08_dc_characters.png" width=1000/>
</div>

|           | mask | cape | tie | ears | smokes | height | class |
|-----------|:----:|:----:|:---:|:----:|:------:|:------:|:-----:|
|   Batman  |   1  |   1  |  0  |   1  |    0   |   180  |  good |
|   Robin   |   1  |   1  |  0  |   0  |    0   |   176  |  good |
|   Alfred  |   0  |   0  |  1  |   0  |    0   |   185  |  good |
|  Penguin  |   0  |   0  |  1  |   0  |    1   |   140  |  evil |
|  Catwoman |   1  |   0  |  0  |   1  |    0   |   170  |  evil |
|   Joker   |   0  |   0  |  0  |   0  |    0   |   179  |  evil |
|  Batgirl  |   1  |   1  |  0  |   1  |    0   |   165  |   ?   |
|  Riddler  |   1  |   0  |  0  |   0  |    0   |   182  |   ?   |
| Your Date |   0  |   1  |  1  |   1  |    1   |   181  |   ?   |

## 🚨 Terminology Alert 🚨: `consistent` and `generalize`

Is this tree __consistent__ with the training examples?
- do all of the training examples get categorized appropriately?

Will this tree __generalize__ well to new examples?
- how well will new examples (test set) perform?

## Great, now how do I build (grow) a tree?

One algorithm that builds a decision tree is called the __ID3 Decision Tree Learning Algorithm__.

It goes like this: 
    
__Main ID3 Loop__:
1. Assign A to be the _best_ decision feature for the next node.
2. Assign A as decision feature for the node
3. For each possible attribute of A, create a new descendant of node
4. Sort training examples to leaf nodes
5. If Training examples are perfectly classified, the STOP, Else, iterate over new leaf nodes.

### What does 'best' mean? How can we tell which node is *best*?

## Selecting a feature: 

<div>
<img src="images/day05_selecting_feature.png" width=600/>
</div>

## Selecting a feature:
__idea__: a good feature splits the examples into __subsets that are as pure as possible__ (ideally) 'all positive' or 'all negative'

> patrons is a better choice--it gives more information about the classification.

## Information Gain:
To select which feature to put at the **root** of the decision tree, we will calculate the **information gain** for each feature, and select the feature with the **largest information gain**.

To calculate information gain, we need to be comfortable with a few concepts:
- Prior Probabilities
- Entropy

## 🚨 Terminology Alert 🚨 `entropy`

__Entropy__: measure of impurity/randomness
- __high entropy__: more evenly split classes - highly unpredictable
- __low entropy__: mostly one class - highly predictable

<div>
<img src="images/day05_entropy.png" width=600/>
</div>

## Calculating Entropy Prior

__Prior Probability__: aka the 'prior'
- the split of the examples into the respective target values.
- if I have 9 positive examples and 5 negative examples, my prior is:

$\langle 9/14, 5/14 \rangle \approx \langle 0.64, 0.36 \rangle$
> the prior probabilities must sum to 1

## Calculating Entropy:

Calculating the entropy when prior is $\langle P_1, ..., P_c\rangle$

<div>
<img src="images/day05_entropy_calc.png" width=600/>
</div>

- entropy of prior $\langle 0.5, 0.5 \rangle$ is $-0.5 \log_{2}(0.5) -0.5 \log_{2}(0.5)  = 1$

- entropy of prior $\langle 0.9, 0.1 \rangle$ is $-0.9 \log_{2}(0.9)-0.1 \log_{2}(0.1) \approx 0.47$

- entropy of prior $\langle 0.64, 0.36 \rangle$ is $-0.64 \log_{2}(0.64)-0.36 \log_{2}(0.36) \approx 0.94$

- entropy of prior $\langle 0.25, 0.25, 0.5 \rangle$ is $-0.25 \log_{2}(0.25)-0.25 \log_{2}(0.25)-0.5 \log_{2}(0.5) \approx 1.5 $

> The maximum entropy is $\log_{2}(k)$ where $k$ is the number of categories. It is not always bounded by 0 and 1

In [ ]:
import math
math.log2(3) # log base 2 of 3 

1.584962500721156

## Code for Entropy:

In [ ]:
import math
# here's the syntax for a log(Base 2)
for i in [.5,1,2,4,8,16,32,64]:
    print("log base 2 of", i, "is", math.log2(i))

log base 2 of 0.5 is -1.0
log base 2 of 1 is 0.0
log base 2 of 2 is 1.0
log base 2 of 4 is 2.0
log base 2 of 8 is 3.0
log base 2 of 16 is 4.0
log base 2 of 32 is 5.0
log base 2 of 64 is 6.0


In [ ]:
import math
def entropy(percentage_list):
    #input: percentage_list consists of float values that sum to 1.0 
    #return: calculation of entropy for input percentages
    result = 0
    for percentage in percentage_list:
        if percentage != 0:
            result += -percentage*math.log2(percentage)
    return result

In [ ]:
#example of a calculation of entropy
entropy([2/6,4/6])

0.9182958340544896

In [ ]:
entropy([1/4, 1/4, 2/4])

1.5

## Calculating Information Gain: 

### Step 1: Entropy *Before*

Start by calculating the entropy of the example __before__ picking a feature:
- $\langle 0.5, 0.5 \rangle = 1$
<div>
<img src="images/day05_patrons_entropy.png" width=600/>
</div>

In [ ]:
#Patrons expected entropy
restaurant[['pat', 'target']].sort_values(['pat', 'target'])

,pat,target
1,Full,No
4,Full,No
8,Full,No
9,Full,No
3,Full,Yes
11,Full,Yes
6,None,No
10,None,No
0,Some,Yes
2,Some,Yes


### Step 2: Calculate the Expected Entropy

The __expected entropy__ for a feature is defined as the weighted sum of the entropies multiplied by the fraction of samples that belong to each set:

Example:

<div>
<img src="images/day05_expected_entropy.png" width=1000/>
</div>

### Step 2: Calculate the Expected Entropy

The **expected entropy** is the amount of entropy we expect after we split our data based on the values of a specific attribute/column in the dataset. 

**Expected Entropy** is calculated as the *weighted sum of the entropies of each partition of the data*.


<div>
<img src="images/day05_patrons_entropy2.png" width=600/>
</div>

In [ ]:
#calculate entropy
entropy_patrons_full = entropy([4/6,2/6]) # 4/6 was No; 2/6 was Yes
entropy_patrons_none = entropy([2/2,0/2])
entropy_patrons_some = entropy([0/4,4/4])
print(entropy_patrons_full, entropy_patrons_none, entropy_patrons_some)

0.9182958340544896 0.0 0.0


In [ ]:
# calculate expected_entropy (weighted average)
expected_entropy_patrons = 6/12*entropy_patrons_full + 2/12*entropy_patrons_none + 4/12*entropy_patrons_some
expected_entropy_patrons

0.4591479170272448

## Information Gain:

The __information gain__ is *difference* between the entropy before the test and the expected entropy after the test. 

$Gain() = Entropy(before) - Expected Entropy (after)$

<div>
<img src="images/day05_IG_calc.png" width=600/>
</div>

In [ ]:
#calculate information gain (prior entropy - expected entropy)
information_gain_patrons = 1.0 - expected_entropy_patrons
information_gain_patrons

0.5408520829727552

# 💬 Exercise:

Calculate the __Information Gain__ for `hun` and `est`:

In [ ]:
restaurant[['hun', 'target']].sort_values(['hun','target'])

,hun,target
4,No,No
6,No,No
8,No,No
10,No,No
2,No,Yes
1,Yes,No
9,Yes,No
0,Yes,Yes
3,Yes,Yes
5,Yes,Yes


# Another Example:

Information Gain Steps:
1. Calculate the entropy before
2. Calculte the expected entropy after
3. Information Gain = Entropy Before - expected entropy after


<div>
<img src="images/entropy_calc.png" width=500/>
</div>

# Decision Tree Considerations

What do we do if we have numeric (even continuous-valued) features like age from teh titanic dataset or petal length from the iris dataset?

- **Idea**: Decision Tree thresholds--`if age > 70`

> Unfortunate Annoying thing: Scikit Learn Decision Tree doesn't work well with categorical data, all predictor variables must be numbers. 

# Does Size Matter? Tree Size Discussion

Both of these trees were made from just 12 examples. Is one better than the other?
<div>
<img src="images/day09_tree_size.png" width=1000/>
</div>

## Tree Size:

There are many different consistent trees possible. 
- Which qualities are preferable? more nodes v fewer nodes
- What are the consequences of having a deep tree with many nodes? 

<div>
<img src="images/bias_variance_tradeoff.png" width=600/>
</div>

## Inductive Bias of ID3

**Shorter trees are preferred in ID3**, trees with **high-information features closer to the root** are preferred.

Baises allow us to learn, but you should understand what your algorithm's bias is. 

## Leaf Nodes (Terminal Nodes)

We don't always have to create a tree that is consistent with the training data. 

Maybe a leaf node that classifies a **majority** of the cases accurately will **generalize** better than a very deep tree. 

Can we **always** make a consistent tree? 

<div>
<img src="images/day09_noisy_data.png" width=600/>
</div>

## Regression with Decision Trees:

How might we go about doing **regression with decision trees**?
- Will Information Gain work for *continuous target variables*?

<div>
<img src="images/day05_selecting_feature.png" width=600/>
</div>

## Regression Example:
<div>
<img src="images/day09_regression.png" width=300/>
</div>
If you had to make a prediction at a leaf node with these groups what would you predict? 
- can't calculate entropy without categorical values

## Example: MPG target values

Which group of examples is more "pure"?
- Group A: 20, 19, 17, 19, 18
- Group B: 14, 17, 24, 7, 30
> Hint: Is there some statistic that we've already talked about that can evaluate the purity of a group of numeric target values?

Standard Deviation.

## Overfitting:

**Big idea:** You **overfit** if you do well on the training set, but not so well on the testing set. 

At what point did this tree 'overfit'?
<div>
<img src="images/day09_overfit.png" width=600/>
</div>

## Avoiding Overfitting:

In general, **big trees overfit the data**. To avoid overfitting, **make the tree smaller**.

Some ideas on avoiding overly complex trees:
- stop growing when data split is not statistically significant
- grow the full tree, the post-prune
- use parameters like `max_depth`, and `num_leaf_nodes` to help

# Decision Trees v kNN:
What are the benefits of deicion trees compared to kNN?
- disadvantages?

When would you use one over the other?

- *If one column highly predicts the target variable:*

- *If lots of predictors have similar weight in the decision:*

- *If you must be albe to interpret the data clearly:*

# Decision Trees v kNN:
What are the benefits of deicion trees compared to kNN?
- disadvantages?

When would you use one over the other?

- *If one column highly predicts the target variable:* **decision tree**

- *If lots of predictors have similar weight in the decision:* **kNN**

- *If you must be albe to interpret the data clearly:* **decision tree**